# install packages

In [ ]:
!pip install llama-index -q
!pip install llama-index-llms-huggingface -q
!pip install llama-index-embeddings-huggingface -q
!pip install transformers bitsandbytes accelerate torch datasets -q

# Import libraries


In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import pandas as pd

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Load the tokenizer and set the pad_token_id


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "NousResearch/Meta-Llama-3-8B-Instruct"
)
tokenizer.pad_token = tokenizer.eos_token 

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids(tokenizer.eos_token),
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
llm = HuggingFaceLLM(
    model_name="NousResearch/Meta-Llama-3-8B-Instruct",
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6, # you can use less temperature for more determinicity e.g. "temperature": 0.25
        "top_p": 0.9,
        "pad_token_id": tokenizer.pad_token_id 
    },
    tokenizer_name="NousResearch/Meta-Llama-3-8B-Instruct",
    stopping_ids=stopping_ids,

    # The quantization line (for T4 GPU)
    # comment it for using TPU
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True},
    device_map="auto",
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Load documents


In [ ]:
documents = SimpleDirectoryReader(input_files=['input 1.pdf', 'input 2.pdf']).load_data()

# Set embedding model and LLM settings


In [ ]:
# index = VectorStoreIndex.from_documents(
#     documents,
#     # llm=llm,
#     # tokenizer=tokenizer,
#     # embed_model="sentence-transformers/all-MiniLM-L6-v2",
#     # embed_model="sentence-transformers/LaBSE",
# )

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/LaBSE")

In [ ]:
# bge embedding model
Settings.embed_model = embed_model

# Llama-3-8B-Instruct model
Settings.llm = llm

# Create index and query engine


In [ ]:
index = VectorStoreIndex.from_documents(
    documents,
)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3)

# Read the "questions 2.xlsx" Excel file


In [ ]:
# Read the Excel file without using the first row as the header
questions_df = pd.read_excel('questions 2.xlsx', header=None)

# Drop the first row which contains irrelevant information
questions_df = questions_df.drop(0)
# questions_df = questions_df.drop(-1)

# Reset the index
questions_df.reset_index(drop=True, inplace=True)

# Set the new first row as the column names
questions_df.columns = questions_df.iloc[0]

# Drop the row that is now used as the header
questions_df = questions_df.drop(0)

# Reset the index again
questions_df.reset_index(drop=True, inplace=True)
questions_df

,ردیف,نام معاونت یا حوزه تخصصی,نام اداره یا زیرمجموعه,عنوان پرسش,پاسخ تفضیلی و تحلیلی,اولویت (اولویت 1، 2 و 3 به ترتیت اهمیت 1 بیشترین اهمیت، 3 کمترین اهمیت),نوع ابزار مورد نیاز طراحی سوال در پرتال ( موشن گرافی، گراف، پادکست و...)
0,1,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,انواع پروانه های ساختمانی را نام ببرید؟,1- پروانه شهرسازی\n 2-پروانه شروع عملیات ساختم...,NaN,NaN
1,2,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,انواع گواهی های ساختمانی را نام ببرید؟,پایان‌کار ساختمان ، تمدید پایان کار ساختمان ، ...,NaN,NaN
2,3,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,مدارک لازم جهت تشکیل پرونده ساختمان و تخریب نو...,1.\tاصل و تصویر سند مالکیت شش دانگ کامل \n2.\t...,1,NaN
3,4,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,مدارک مورد نیاز برای پیشنویس پروانه,1.\tبرگه های تعهد مهندسان ناظر و طراح ، معماری...,NaN,NaN
4,5,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,مدارک مورد نیاز برای پیشنویس تغییر نقشه و مهند...,1.\tداشتن رضایت نامه از همه مهندسان (ناظر، معم...,1,NaN
...,...,...,...,...,...,...,...
80,81,شهرسازی و معماری,اداره کل امور اجرایی ماده صد,آیا املاکی که دارای رای کمیسیون ماده صد میباشن...,براساس رای هیئت عمومی دیوان عدالت اداری ، اخذ ...,2,NaN
81,82,شهرسازی و معماری,اداره کل امور اجرایی ماده صد,آیا امکان ایجاد یک واحد تجاری در واحد مسکونی (...,بر اساس تبصره ذیل بند 24 ماده 55 قانون شهرداری...,2,NaN
82,83,شهرسازی و معماری,NaN,برای قطع درختان خشك به چه نحوی باید عمل كرد ؟,سوال مطرح شده مرتبط با حوزه شهرسازی و معماری ن...,2,NaN
83,84,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,نحوه واگذاری گذر فی مابین پلاک های تجمیعی چگون...,در صورتیکه املاک تجمیعی در بافت فرسوده قرار د...,1,NaN


In [ ]:
questions_df['عنوان پرسش'][0]

'انواع پروانه های ساختمانی را نام ببرید؟'

In [ ]:
response = query_engine.query(questions_df['عنوان پرسش'][0])
print(response)

 According to the provided context information, the types of building permits mentioned are:
1. پروانه مسكوني (Residential Permit)
2. پروانه تجاري (Commercial Permit)
3. پروانه اداري (Administrative Permit)

Note that the query asks for the types of building permits, which are explicitly mentioned in the provided context information. The answer is a list of the mentioned permit types.


# inference with RAG

In [16]:
responses = []
for question in tqdm(questions_df['عنوان پرسش'], desc="Processing questions"):
    try:
        response = query_engine.query(question)

        # Use the response string directly
        combined_docs_text = response

        # input_text = f"Question: {question}\nContext: {combined_docs_text}\nAnswer:"
        input_text = f"Question: {question}\nContext: {combined_docs_text}\nPlease answer in Persian.\nAnswer:"

        # generated_response = llm.generate(input_text)
        # generated_response = llm.predict(input_text)
        # generated_response = llm.completion_to_prompt(input_text)
        # generated_response = llm.chat(input_text)
        generated_response = llm.complete(input_text)

        # Append the response to the list
        responses.append(generated_response)
    except Exception as e:
        print(f"Error processing question: {question}")
        print(f"Exception: {e}")
        responses.append("Error generating response")

Processing questions: 100%|██████████| 85/85 [49:31<00:00, 34.96s/it]

Error processing question: nan
Exception: 'float' object has no attribute 'query_str'


In [17]:
questions_df['response'] = responses
questions_df.to_excel('responses.xlsx', index=False)

In [18]:
questions_df

,ردیف,نام معاونت یا حوزه تخصصی,نام اداره یا زیرمجموعه,عنوان پرسش,پاسخ تفضیلی و تحلیلی,اولویت (اولویت 1، 2 و 3 به ترتیت اهمیت 1 بیشترین اهمیت، 3 کمترین اهمیت),نوع ابزار مورد نیاز طراحی سوال در پرتال ( موشن گرافی، گراف، پادکست و...),response
0,1,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,انواع پروانه های ساختمانی را نام ببرید؟,1- پروانه شهرسازی\n 2-پروانه شروع عملیات ساختم...,NaN,NaN,"پروانه مسكوني، تجاري يا اداري. (Residential, ..."
1,2,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,انواع گواهی های ساختمانی را نام ببرید؟,پایان‌کار ساختمان ، تمدید پایان کار ساختمان ، ...,NaN,NaN,پروانه مسكوني، پروانه تجاري، پروانه اداري.
2,3,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,مدارک لازم جهت تشکیل پرونده ساختمان و تخریب نو...,1.\tاصل و تصویر سند مالکیت شش دانگ کامل \n2.\t...,1,NaN,مدارک لازم جهت تشکیل پرونده ساختمان و تخریب ن...
3,4,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,مدارک مورد نیاز برای پیشنویس پروانه,1.\tبرگه های تعهد مهندسان ناظر و طراح ، معماری...,NaN,NaN,در مورد پیشنویس پروانه، مدارک مورد نیاز شامل ...
4,5,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,مدارک مورد نیاز برای پیشنویس تغییر نقشه و مهند...,1.\tداشتن رضایت نامه از همه مهندسان (ناظر، معم...,1,NaN,در مورد پیشنویس تغییر نقشه و مهندس ناظر، مدار...
...,...,...,...,...,...,...,...,...
80,81,شهرسازی و معماری,اداره کل امور اجرایی ماده صد,آیا املاکی که دارای رای کمیسیون ماده صد میباشن...,براساس رای هیئت عمومی دیوان عدالت اداری ، اخذ ...,2,NaN,بلی، املاکی که دارای رای کمیسیون ماده صد میبا...
81,82,شهرسازی و معماری,اداره کل امور اجرایی ماده صد,آیا امکان ایجاد یک واحد تجاری در واحد مسکونی (...,بر اساس تبصره ذیل بند 24 ماده 55 قانون شهرداری...,2,NaN,بلی، اگر شرایط مشخص شده در متن رعایت شود، امک...
82,83,شهرسازی و معماری,NaN,برای قطع درختان خشك به چه نحوی باید عمل كرد ؟,سوال مطرح شده مرتبط با حوزه شهرسازی و معماری ن...,2,NaN,برای قطع درختان خشك باید با حفد درختدان و ایج...
83,84,شهرسازی و معماری,تدوین ضوابط، نظارت و صدور پروانه,نحوه واگذاری گذر فی مابین پلاک های تجمیعی چگون...,در صورتیکه املاک تجمیعی در بافت فرسوده قرار د...,1,NaN,نحوه واگذاری گذر فی مابین پلاک های تجمیعی در ...


# Read the "questions 1.xlsx" Excel file


In [30]:
questions_df_1 = pd.read_excel('questions 1.xlsx')
questions_df_1

,پاسخ,سوال
0,با سلام و احترام؛ لطفا به شهرداری منطقه مراجعه...,باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبت...
1,NaN,NaN
2,با سلام و احترام؛ صدور پروانه صرفا درچهارچوب ض...,با سلام احترام خدمت شما من سال پیش در منطقه ۱۴...
3,NaN,NaN
4,با سلام و احترام؛ در نسخه دیجیتالی اخذ پرینت ...,چگونه می توانم از صورتجلسات کمیسیون ماه 5 پرین...
...,...,...
108,با سلام و احترام ارائه اجاره نامه یا سند مالکی...,با سلام در خصوص حقوق کسبی ملک ارایه فیش برق تج...
109,NaN,NaN
110,با سلام و احترام الزامی نمی باشد.,با سلام باتوجه به اینکه قبوض وابلاغیه ها به صو...
111,NaN,NaN


In [31]:
questions_df_1 = pd.DataFrame(questions_df_1.dropna())
questions_df_1.reset_index(drop=True, inplace=True)
questions_df_1

,پاسخ,سوال
0,با سلام و احترام؛ لطفا به شهرداری منطقه مراجعه...,باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبت...
1,با سلام و احترام؛ صدور پروانه صرفا درچهارچوب ض...,با سلام احترام خدمت شما من سال پیش در منطقه ۱۴...
2,با سلام و احترام؛ در نسخه دیجیتالی اخذ پرینت ...,چگونه می توانم از صورتجلسات کمیسیون ماه 5 پرین...
3,با سلام و احترام تا این تاریخ شیوه نامه مذکور ...,باسلام اینجانب زمین فریز شده در منطقه ۱۴ فریز ...
4,با سلام و احترام؛ هرساختمانی که احداث می شود...,با سلام مجدد بنده از همکاران شهرداری میباشم قب...
5,با سلام و احترام؛ خیر صدور پروانه صرفا درچارچو...,با سلام.ما زمینی ۳۰۲ متری در منطقه ۵ تهران دار...
6,با سلام و احترام؛ لطفا شماره پرونده ومنطقه مرب...,با سلام و احترام.اینجانب پس از دریافت دستور نق...
7,با سلام و احترام؛ باید مشخصات ملک ازجمله شماره...,با سلام مجدد این سومین پیام بنده میباشد قبلا ه...
8,با سلام و احترام؛ صدور پروانه درچارچوب ضوابط...,با سلام می خواستم بپرسم که ایا درسته که احتمال...
9,با سلام و احترام مرجع تشخیص باغ ، شورای اسلامی...,با سلام ملک اینجانب در سال ۸۶توسط کمیسیون ماده...


In [32]:
questions_df_1['سوال'][0]

'باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبتی منطقه 5 شهرداری تهران ناحیه 3 مراداباد می باشم درخواست جواز ساخت را از ان مقامات مجترم تقاضامندم با توجه به اینکه پلاک فوق بمرور زمان در داخل شهر تهران قرارگرفته لذا تقاضامندم بزرگواران در این مورد دستورات لازم را صادر وامر به ابلاغ فرمایند با تقدیم احترامات فایقه رحیم جوادی'

In [33]:
response = query_engine.query(questions_df_1['سوال'][0])
print(response)

 با سلام، به استناد ماده 11-3 قانون برنامه و بودجه کشور، شهرداری تهران موظف است تا حداکثر ظرف مدت 2 سال، پس از ابلاغ طرح تفصیلی، نسبت به شناسایی و ثبت جداره‌های واجد ارزش تاریخی، اقدام نماید. با توجه به اینکه پلاک ثبتی منطقه 5 شهرداری تهران ناحیه 3 در داخل شهر تهران قرار گرفته، بنابراین تقاضانمندان باید از طریق کمیته اجرایی موضوع مصوبه مندرج در بند 2-13، با همکاري سازمان میراث فرهنگی، گردشگری و صنایع دستی، نسبت به اخذ مجوز ساخت اقدام نمایند. با تقدیم احترامات فائقه رحیم جوادی. 
---------------------
Note: The query is a request for a building permit from the authorities, and the answer is a response from the authorities, citing the relevant law and regulations. The answer is in Persian, the official language of Iran.


In [35]:
response = query_engine.query(questions_df_1['سوال'][0])
combined_docs_text = response
input_text = f"Question: {question}\nContext: {combined_docs_text}\nPlease answer in Persian.\nAnswer:"
generated_response = llm.complete(input_text)
generated_response

CompletionResponse(text=' با سلام، با توجه به اطلاعات شما، در مورد پرونده ۱۴۰۱۰۲۷۵۶، متاسفانه بعد از بررسی، متوجه شدیم که پرونده شما در مرحله تعلیق و درحال بررسی است و تاکنون تصمیمی گرفته نشده است. به همین دلیل، نمی\u200cتوانیم درباره برگشت به روال قبل صحبت کنیم. درصورتی که پس از بررسی و تصمیم\u200cگیری نهایی، نتیجه پرونده شما اعلام می\u200cشود. در این مورد، توصیه می\u200cکنیم تا صبر و حوصله داشته باشید و منتظر نتیجه نهایی باشید. متاسفانه، نمی\u200cتوانیم درباره امیدی یا گرفتار بودن صحبت کنیم. \n\nTranslation: Hello, based on the information you provided, regarding case ۱۴۰۱۰۲۷۵۶, unfortunately, after reviewing, we found that your case is in the process of being reviewed and no decision has been made yet. Therefore, we cannot discuss returning to the previous process. In this case, we recommend that you be patient and wait for the final result. Unfortunately, we cannot discuss hope or being trapped. \n\nNote: The answer is a formal response to the query, citing the case number and prov

In [38]:
generated_response.text

' با سلام، با توجه به اطلاعات شما، در مورد پرونده ۱۴۰۱۰۲۷۵۶، متاسفانه بعد از بررسی، متوجه شدیم که پرونده شما در مرحله تعلیق و درحال بررسی است و تاکنون تصمیمی گرفته نشده است. به همین دلیل، نمی\u200cتوانیم درباره برگشت به روال قبل صحبت کنیم. درصورتی که پس از بررسی و تصمیم\u200cگیری نهایی، نتیجه پرونده شما اعلام می\u200cشود. در این مورد، توصیه می\u200cکنیم تا صبر و حوصله داشته باشید و منتظر نتیجه نهایی باشید. متاسفانه، نمی\u200cتوانیم درباره امیدی یا گرفتار بودن صحبت کنیم. \n\nTranslation: Hello, based on the information you provided, regarding case ۱۴۰۱۰۲۷۵۶, unfortunately, after reviewing, we found that your case is in the process of being reviewed and no decision has been made yet. Therefore, we cannot discuss returning to the previous process. In this case, we recommend that you be patient and wait for the final result. Unfortunately, we cannot discuss hope or being trapped. \n\nNote: The answer is a formal response to the query, citing the case number and providing information about 

# inference with RAG

In [40]:
responses = []
for question in tqdm(questions_df_1['سوال'], desc="Processing questions"):
    try:
        response = query_engine.query(question)

        # Use the response string directly
        combined_docs_text = response

        # input_text = f"Question: {question}\nContext: {combined_docs_text}\nAnswer:"
        input_text = f"Question: {question}\nContext: {combined_docs_text}\nPlease answer in Persian.\nAnswer:"

        generated_response = llm.complete(input_text)
        # print(generated_response)

        # Append the response to the list
        responses.append(generated_response)
    except Exception as e:
        print(f"Error processing question: {question}")
        print(f"Exception: {e}")
        responses.append("Error generating response")

Processing questions: 100%|██████████| 56/56 [45:37<00:00, 48.89s/it]


In [41]:
questions_df_1['پاسخ مدل'] = responses
questions_df_1.to_excel('responses_1.xlsx', index=False)

In [42]:
questions_df_1

,پاسخ,سوال,پاسخ مدل
0,با سلام و احترام؛ لطفا به شهرداری منطقه مراجعه...,باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبت...,با سلام و احترام، ملاحظه می‌کنم که شما دارای ...
1,با سلام و احترام؛ صدور پروانه صرفا درچهارچوب ض...,با سلام احترام خدمت شما من سال پیش در منطقه ۱۴...,با سلام، متاسفانه درک نکردم که درباره چه مورد...
2,با سلام و احترام؛ در نسخه دیجیتالی اخذ پرینت ...,چگونه می توانم از صورتجلسات کمیسیون ماه 5 پرین...,برای دریافت پرینت از صورتجلسات کمیسیون ماده 5...
3,با سلام و احترام تا این تاریخ شیوه نامه مذکور ...,باسلام اینجانب زمین فریز شده در منطقه ۱۴ فریز ...,با سلام، متاسفانه سایت شما در مورد لغو شیوه ن...
4,با سلام و احترام؛ هرساختمانی که احداث می شود...,با سلام مجدد بنده از همکاران شهرداری میباشم قب...,با سلام مجدد، بنده از همکاران شهرداری میباشم....
5,با سلام و احترام؛ خیر صدور پروانه صرفا درچارچو...,با سلام.ما زمینی ۳۰۲ متری در منطقه ۵ تهران دار...,با سلام. با توجه به ماده ۱۴ قانون زمین شهری، ...
6,با سلام و احترام؛ لطفا شماره پرونده ومنطقه مرب...,با سلام و احترام.اینجانب پس از دریافت دستور نق...,با سلام و احترام. متاسفانه، در خصوص درخواست ش...
7,با سلام و احترام؛ باید مشخصات ملک ازجمله شماره...,با سلام مجدد این سومین پیام بنده میباشد قبلا ه...,با سلام مجدد، در پاسخ به سوال شما، بیمارستان ...
8,با سلام و احترام؛ صدور پروانه درچارچوب ضوابط...,با سلام می خواستم بپرسم که ایا درسته که احتمال...,با سلام، با توجه به متن، احتمال احداث طبقه ۶ ...
9,با سلام و احترام مرجع تشخیص باغ ، شورای اسلامی...,با سلام ملک اینجانب در سال ۸۶توسط کمیسیون ماده...,با سلام، با توجه به اطلاعات ارائه شده، در مور...
